# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

Для начала загрузим и изучим данные.

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Посмотрим общую информацию о датасете.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Проверим есть ли дубликаты.

In [ ]:
df.duplicated().sum()

153

Удалим дубликаты

In [ ]:
df = df.drop_duplicates()
df.shape

(4847, 5)

Посмотрим какие данные содержат категориальные признаки.

In [ ]:
print('Распределения признаков пола:')
print(df['Пол'].value_counts())
print()
print('Распределение признаков "члены семья"')
print(df['Члены семьи'].value_counts())
print()
print('Распределение признаков "Страховые выплаты"')
print(df['Страховые выплаты'].value_counts())

Распределения признаков пола:
0    2431
1    2416
Name: Пол, dtype: int64

Распределение признаков "члены семья"
1    1748
0    1461
2    1038
3     437
4     124
5      32
6       7
Name: Члены семьи, dtype: int64

Распределение признаков "Страховые выплаты"
0    4284
1     423
2     114
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64


### Выводы:

- Датасет состоит из 5 столбцов. 
- Пропусков и выбросов нет. 
- В процессе первичного изучения датасета удалили 153 дубликата. 
- Категориальные данные без видимых ошибок.
- Количество мужчин и женщин в датасете примерно одинаковое.

## Умножение матриц

### Качество исходной модели

Разделим наш датасет на обучающую и валидационную выборку.

In [ ]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Обучим модель линейной регрессии и вычислим качество модели.

In [ ]:
model = LinearRegression() # используем модель линейной регрессии
model.fit(features_train, target_train) # обучим модель на тренировочной выборке
predicted_valid = model.predict(features_valid) # предскажем целевой признак валидационной выборки

r2_original = model.score(features_valid, target_valid) # определим качество модели
r2_original

0.42307727492147584

### Создание матрицы

Умножим признаки датасета на обратимую матрицу и проверим изменится ли качество линейной регрессии.

Обратимая матрица должна быть квадратной, быть размером с 4 х 4 (количество признаков в датасете). У нашей матрицы должна быть противоположная матрица при умножении на которую получатся 1 по главной диагонале.

Создадим случайную обратимую
матрицу.

In [ ]:
matrix = np.random.randn(4, 4)
matrix

array([[ 1.16599548,  0.67674978,  1.98850426, -0.07799509],
       [-0.85776141,  0.59147643, -0.1374552 , -0.08799634],
       [ 0.96608835, -1.10808501,  0.94853487, -0.40658627],
       [ 0.04435489,  0.51089887, -0.45857879,  0.56562344]])

Проверим получившеюся матрицу на обратимость. Для этого умножим её на обратную ей матрицу. В результате должна получиться единичная матрица.

In [ ]:
np.round(np.dot(matrix, np.linalg.inv(matrix)))

array([[ 1.,  0., -0., -0.],
       [-0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [-0.,  0.,  0.,  1.]])

Проверим матрицу на обратимость с помощью определителя квадратной матрицы.

In [ ]:
np.linalg.det(matrix)

0.26449790465268125

Определитель не равен 0, значит матрица обратимая.

### Умножение обратной матрицы на датафрейм.

Преобразуем датафрейм признаков в матрицу.

In [ ]:
features_train_matrix = features_train.values
features_valid_matrix = features_valid.values

Перемножим тестовую и тренировочную матрицы на нашу обратимую матрицу

In [ ]:
features_train_matrix = np.dot(features_train_matrix, matrix)
features_valid_matrix = np.dot(features_valid_matrix, matrix)

In [ ]:
features_train_matrix

array([[ 52627.88628362, -60373.0497866 ,  51690.38425138,
        -22160.28411141],
       [ 48949.29459278, -56156.24614174,  48087.02429455,
        -20616.77988509],
       [ 27029.93178771, -31011.6738848 ,  26555.21872257,
        -11385.96171813],
       ...,
       [ 27904.00834083, -32010.63954776,  27411.43842635,
        -11751.61536665],
       [ 37457.1320745 , -42972.99193123,  36800.14667442,
        -15777.96334591],
       [ 45660.89928929, -52387.14853687,  44859.14630413,
        -19234.00695346]])

### Обучение измененных данных

Переводим получившуюся матрицу в датафрейм и обучаем моделью линейная регрессия.

In [ ]:
converted_features_train = pd.DataFrame (features_train_matrix, columns = features_train.columns)
converted_features_valid = pd.DataFrame (features_valid_matrix, columns = features_valid.columns)
converted_features_valid.head()

,Пол,Возраст,Зарплата,Члены семьи
0,45580.807446,-52287.331660,44769.672673,-19192.973635
1,63638.939676,-73003.198018,62506.037556,-26796.928764
2,32723.343531,-37542.864494,32151.867256,-13785.125022
3,18612.141105,-21361.951272,18300.444988,-7849.415531
4,26528.061218,-30444.108000,26077.468697,-11184.038882


In [ ]:
model = LinearRegression()
model.fit(converted_features_train, target_train)
predicted_valid = model.predict(converted_features_valid)

r2_converted = model.score(converted_features_valid, target_valid)
print('R2 исходной модели', r2_original)
print('R2 трансформированной модели', r2_converted)

R2 исходной модели 0.42307727492147584
R2 трансформированной модели 0.42307727491908564


Как видим, качество обучения линейной регрессии не меняет при умножении признаков на обратимую матрицу. 

### Обоснование равенства

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $X_1$ — матрица признаков умноженная на обратимую матрицу

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<b>Докажем, что качество линейной регрессии не изменится при умножении признаков на обратимую матрицу. </b>

Формула матрицы признаков умноженная на обратимую матрицу выглядит следующим образом:


$$
X_1 = XP
$$

Предсказание для трансформированной матрицы выглядит следующим образом:

$$
a_1 = X_1w_1
$$

Где $w_1$ - вектор весов для линейной регрессии для матрицы $X_1$

$$
w_1 = (X_1^T X_1)^{-1} X_1^T y
$$

Заменим $w_1$ в формуле предсказания:

$$
a_1 = X_1(X_1^T X_1)^{-1} X_1^T y
$$

Заменим $X_1$ на XP:

$$
a_1 = XP((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки, воспользовавшись свойством $(AB)^T = B^TA^T$, а потом свойством $(ABС)^{-1} = С^{-1}B^{-1}A^{-1} $:

$$
a_1 = XP((XP)^T (XP))^{-1} (XP)^T y = XP(P^T X^TX P)^{-1} P^T X^T y = XP(P)^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Произведение матрицы на обратную ей матрицу  $PP^{-1}$ и $(P^T)^{-1}P^T$ равна еденичной матрице E:

$$
a_1 = XE(X^TX)^{-1}EX^Ty
$$

Поскольку если любую матрицу умножить на единичную, то получится эта же матрица, убираем из формулы E:

$$
a_1 = X(X^TX)^{-1}X^Ty
$$

Формула обучения $(X^T X)^{-1} X^T y = w $, заменим часть нашей формулы на $w$:

$$a^1 = Xw = a $$

Как видно, значение предсказания  не меняется, если умножать матрицу признаков на обратимую матрицу.

## Алгоритм преобразования

В качестве защиты персональных данных при работе с линейной регрессией можно использовать умножение матрицы признаков на обратимую матрицу, которая будет генерироваться случайным образом.

Алгоритм:

- Генерация матрицы.
- Проверка матрицы на обратимость.
- Умножение матрицы признаков на сгенерированную обратимую матрицу.
- Применение модели линейной регрессии к измененной матрице признаков.